In [1]:
from isotools import Transcriptome
from isotools import __version__ as isotools_version
from isotools._utils import _filter_event

import os
from pathlib import Path
import logging
from collections import Counter
from urllib.request import urlretrieve
import itertools
from tqdm.notebook import tqdm_notebook as tqdm

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
logger=logging.getLogger('isotools')
logger.info(f'This is isootools version {isotools_version}')

INFO:This is isootools version 0.2.11rc1


In [2]:
try:
    isoseq=Transcriptome.load("./theco_isotools.pkl")
    
except FileNotFoundError:    
    try:
        isoseq=Transcriptome.from_reference(anno+".isotools.pkl")
        
    except:
        isoseq=Transcriptome.from_reference(anno+".gtf.gz")
        isoseq.save_reference(anno+'.isotools.pkl')

INFO:loading transcriptome from ./theco_isotools.pkl


In [3]:
stbl=isoseq.sample_table

In [4]:
tissue_names=list(set(stbl[stbl.sample_type=="tissue"].sample_subtype))

In [5]:
groups={}

for tissue in tissue_names:
    groups[tissue]=list(stbl.name[stbl.sample_subtype==tissue])

In [9]:
genes=isoseq.gene_table()

In [18]:
genes[genes.n_transcripts>100]

,chr,start,end,strand,gene_name,n_transcripts
23,chr22_KI270734v1_random,138081,161852,-,ENSG00000277196.4,328
64,chr21,37073225,37203112,+,ENSG00000182670.13,1556
199,chr21,45263927,45287898,-,ENSG00000186866.16,400
517,chr21,46286336,46297751,+,ENSG00000182362.13,111
520,chr21,41885111,41953890,-,ENSG00000157617.16,146
...,...,...,...,...,...,...
694291,chr22_KI270733v1_random,122922,178625,-,PB_novel_51331,118
694635,chrUn_KI270742v1,315,110003,-,PB_novel_51469,258
694701,SIRV5,1000,13606,+,PB_novel_51499,329
694708,SIRV6,999,11838,+,PB_novel_51502,183


In [20]:
g=isoseq["ENSG00000182670.13"]

In [25]:
g.data.keys()

dict_keys(['ID', 'chr', 'strand', 'name', 'reference', 'transcripts', 'segment_graph', 'coverage', 'chimeric'])

In [32]:
sg=g.segment_graph

In [34]:
sg.alternative_splicing_events()

AttributeError: 'SegmentGraph' object has no attribute 'alternative_splicing_events'

In [38]:
isoseq.add_filter( "PROTEIN_CODING", 'transcript_type=="protein_coding"', context='reference')

In [69]:
res=isoseq.alternative_splicing_events(progress_bar=True,query="not NOVEL_GENE",samples=groups["aorta"])

100%|███████████████████████████████| 695092/695092 [16:34<00:00, 698.61genes/s]


In [64]:
sum(res.iloc[:,6])

14920

In [68]:
res[res.splice_type=="IR"]

,gene,chr,start,end,splice_type,novel,aorta_ENCFF745HHL_in_cov,aorta_ENCFF099TJN_in_cov,aorta_ENCFF745HHL_total_cov,aorta_ENCFF099TJN_total_cov
29,ENSG00000159079.18,chr21,32601973,32602271,IR,False,80,6,92,11
30,ENSG00000159079.18,chr21,32602367,32603160,IR,False,79,6,91,10
37,ENSG00000160310.17,chr21,46635763,46636443,IR,True,76,7,92,11
64,ENSG00000157557.11,chr21,38814392,38814780,IR,True,71,77,86,79
